# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. Prakash Prasad
2. HARSHAV KUMAR
3. SAMOSOUMO BARAL
4. GOURAV KALRA

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it


Coding begins here

### 1.	Define the environment in the following block

The start point, destination, vegetation and other obstacles, radar interference as a 2D grid (list of list)
Current drone position and prob of enemy and radar detection in each cell of the grid


List the PEAS decription of the problem here in this markdown block

-1. PEAS:
-* Defining minimal possible requirements our system should have to solve the problem in given setting*

Enviornment - The start point, destination, vegetation and other obstacles, radar interference as a 2D grid
Performamce measure - Number of cells it takes to get to destination undetected. (Better measure: Route length/ straight line dist. bw start and dest.)
Sensors - Satelite signal detectors, Radar detector, user signal detector, visual/sonic input to identify vegetation laid no fly zones
Actuators - Motor control to allow drone movement, turning and altitude change.

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

Data Structures

Priority Queue for open list
List for closed list
numpy array for grid: each cell with heuristic function values

In [1]:
#Code Block : Set Initial State (Must handle dynamic inputs)
import random
import numpy as np


def set_initial_state(all_initial_states, k, seed_=100):
    # Selecting k random states from available set of initial states
    random.seed(seed_)
    initial_state_list = random.sample(all_initial_states, k)
    return initial_state_list

In [2]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

# For simplicity, we do not vary the size of the grid. It's a 10 x 10 square matrix and from any point we can go to all 8 neighbours as long as index in [0,9] and it is not a red cell
def h(PoD, PoM): return (1 + PoD) * (1 + PoM) if PoD != -1 else np.inf
# evaluation function as defined in problem statement


In [3]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented
def get_successor_states(curr_state, transition_matrix):
    # curr_state should be of the form (row_index, col_index)
    r, c = curr_state
    possible_successors = [[r+1,c], [r-1,c], [r,c+1], [r, c-1], [r+1, c+1], [r-1,c-1], [r+1,c-1], [r-1,c+1]]
    pre_veg_successors = [i for i in possible_successors if (0<=i[0]<10) and (0<=i[1]<10)]       # removing out of grid elements
    successor_state_list = [ i for i in pre_veg_successors if transition_matrix[i[0]][i[1]] != np.inf]       # removing red cells
    return successor_state_list

# get_successor_states((6,0),transition_matrix), get_successor_states((4,2),transition_matrix)


In [4]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented
def check_goal_state(curr_state_list, goal): return True if goal in curr_state_list else False

### 2.Definition of Algorithm 1 (Local Beam search, k=3) Use the above-mentioned algorithm and implement in PYTHON

In [5]:
# #Code Block : Function for algorithm 1 implementation
def make_key(state): return str(state[0])+str(state[1])


def local_beam_search(start_list, goal, k, transition_matrix, seed=100):

    inital_states_list = set_initial_state(start_list, k, seed)     # get k random states from given start states
    print('inital states randomised with given k=', k, ' : ', inital_states_list)
    print('goal_state: ', goal)

    goal_flag = check_goal_state(inital_states_list, goal)      # check if goal is in one of these start states
    path= dict(zip([make_key(i) for i in inital_states_list], [[make_key(i),0] for i in inital_states_list])) if not goal_flag \
        else {make_key(goal): [make_key(goal), 0]}

    open_list = [[i, transition_matrix[i[0]][i[1]]] for i in inital_states_list]
    open_list = sorted(open_list, key=lambda x: x[1])       # sort priority queue
    closed_list = []

    while (open_list != []) and (goal_flag == False):
        curr_node = open_list[0][0]     # pop the first element of open list, put it in closed list and expand
        open_list = open_list[1:]
        closed_list.append(curr_node)

        successsor_state_list  = get_successor_states(curr_node, transition_matrix)

        for node in successsor_state_list:
            state_key , n_key = make_key(curr_node), make_key(node)

            # 3 cases are possible - either the node is totally unexplored - add in open list
            if (node not in open_list) and (node not in closed_list):
                open_list.append([node, transition_matrix[node[0]][node[1]]])       #append it in open list
                # path[n_key] = [state_key, path[state_key][1] + 1]
                if not n_key in path.keys():
                    path[n_key] = [state_key, path[state_key][1]+1]   # if the node is being reached first time
                else:
                    existing_cost, new_cost = path[n_key][1], path[state_key][1] + 1
                    path[n_key] = [state_key, new_cost] if new_cost <= existing_cost else path[n_key]
            # if its present in open list but not in closed list
            elif (node in open_list) and (node not in closed_list):
                existing_cost, new_cost = path[n_key][1], path[state_key][1]+1
                print(existing_cost, new_cost )
                path[n_key] = [state_key, new_cost] if new_cost<=existing_cost else path[n_key]      # incase this new path gives lower cost, change parents
            # nodes cant both be in open and closed list at the same time
            elif (node in open_list) and (node in closed_list):
                print('node n in open and closed list', node)   # Sanity check
                print(open_list)
                print(closed_list)
                raise AssertionError

        open_list = sorted(open_list, key=lambda x: x[1])
        goal_flag = check_goal_state([x[0] for x in  open_list], goal)
        open_list = open_list[:k]

    return goal_flag, path, inital_states_list


## 3. DYNAMIC INPUT
IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [6]:
#Code Block : Function & call to get inputs (start/end state)

# [todo] imp: Give random incorrect input to use default input from question

# 00,01,02,10,20
# 9_6
def get_user_inputs():
    # to get user input on start state and end state

    try:
        start_list_ = input("Enter the list of start states (current location) format comma seperated rowindexcolindex eg 01,02,11: ").split(',')
        start_list = [[int(s[0]), int(s[1])] for s in start_list_]
        goal = list(map(int, input("Enter the goal state (destination) format rowindex_colindex eh=g 9_6: ").split('_')))
        k = int(input("Parameter k of the local beam search: "))
    except:
        start_list = [[1, 0], [0, 0], [0, 1], [0,2], [2,0]]
        goal = [9, 6]
        k = 3

    return start_list, goal, k

# matrix example as provided in question
transition_matrix = np.array([
    [h(0, 0), h(0, 0), h(0, 0), h(-1, -1), h(-1, -1), h(-1, -1), h(0.5, 0.5), h(0.5, 0.5), h(-1, -1), h(-1, -1)],
    [h(0, 0), h(0, 0), h(0, 0), h(0.5, 0.5), h(0.5, 0.5), h(-1, -1), h(0.5, 0.5), h(0.5, 0.5), h(-1, -1),  h(0.5, 0.5)],
    [h(0, 0), h(0, 0), h(0, 0), h(0.5, 0.5), h(0.5, 0.5), h(0.5, 0.5), h(-1, -1), h(0.5, 0.5), h(0.5, 0.5), h(0.5, 0.5)],
    [h(0.2, 0), h(0.2, 0.1), h(0.2, 0.1), h(0.5, 0.5), h(0.5, 0.5), h(0.5, 0.5), h(-1, -1), h(0.5, 0.5), h(0.5, 0.5), h(0.5, 0.5)],
    [h(0.2, 0.9), h(0.2, 1), h(0.2, 0.1), h(0.3, 0.9), h(0.3, 0.9), h(0.2, 0.1), h(-1, -1), h(-1, -1), h(-1, -1), h(0.5, 0.5)],
    [h(0.2, 0.9), h(-1, -1), h(0.2, 0.1), h(-1, -1), h(0.3, 0.9), h(0.2, 0.1), h(0.2, 0.1), h(0.2, 0.9), h(0.2, 0.9), h(0.2, 0.1)],
    [h(0.2, 0.9), h(-1, -1), h(0.2, 0.1), h(-1, -1), h(0.2, 0.1), h(0.3, 0.9), h(0.2, 0.1), h(0.2, 0.1), h(0.2, 0.1), h(0.2, 0.1)],
    [h(0.2, 0.9), h(-1, -1), h(0.2, 0.1), h(-1, -1), h(0.2, 0.1), h(-1, -1), h(-1, -1), h(-1, -1), h(0.2, 0.1), h(0.2, 0.1)],
    [h(-1, -1), h(0.05, 0.05), h(0.05, 0.05), h(-1, -1), h(0.8, 0.9), h(0.8, 0.9), h(0.05, 0.05), h(0.05, 0.05), h(0.2, 0.1), h(0.2, 0.1)],
    [h(0.2, 0.9), h(0.8, 1), h(0.05, 0.05), h(0.8, 0.9), h(0.8, 0.9), h(-1, -1), h(0, 0), h(0.05, 0.05), h(0.2, 0.1), h(-1, -1)]
])


start_list, goal, k = get_user_inputs()
print(start_list, goal, k)
print(type(start_list), type(goal))

[[1, 0], [0, 0], [0, 1], [0, 2], [2, 0]] [9, 6] 3
<class 'list'> <class 'list'>


In [7]:
transition_matrix

array([[1.    , 1.    , 1.    ,    inf,    inf,    inf, 2.25  , 2.25  ,
           inf,    inf],
       [1.    , 1.    , 1.    , 2.25  , 2.25  ,    inf, 2.25  , 2.25  ,
           inf, 2.25  ],
       [1.    , 1.    , 1.    , 2.25  , 2.25  , 2.25  ,    inf, 2.25  ,
        2.25  , 2.25  ],
       [1.2   , 1.32  , 1.32  , 2.25  , 2.25  , 2.25  ,    inf, 2.25  ,
        2.25  , 2.25  ],
       [2.28  , 2.4   , 1.32  , 2.47  , 2.47  , 1.32  ,    inf,    inf,
           inf, 2.25  ],
       [2.28  ,    inf, 1.32  ,    inf, 2.47  , 1.32  , 1.32  , 2.28  ,
        2.28  , 1.32  ],
       [2.28  ,    inf, 1.32  ,    inf, 1.32  , 2.47  , 1.32  , 1.32  ,
        1.32  , 1.32  ],
       [2.28  ,    inf, 1.32  ,    inf, 1.32  ,    inf,    inf,    inf,
        1.32  , 1.32  ],
       [   inf, 1.1025, 1.1025,    inf, 3.42  , 3.42  , 1.1025, 1.1025,
        1.32  , 1.32  ],
       [2.28  , 3.6   , 1.1025, 3.42  , 3.42  ,    inf, 1.    , 1.1025,
        1.32  ,    inf]])

### 4.	Calling the search algorithms - Print the optimal path sequence with costs
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [8]:
def get_traversed_path(path, goal):
    g = make_key(goal)
    traversed_keys = [g]

    key = g
    num_iter = 0
    while (key != path[key][0] ):
        temp = path[key][0]
        traversed_keys.append(temp)
        key = temp
        num_iter +=1
    traversed_path = [[int(k[0]), int(k[1])] for k in traversed_keys[::-1]]

    return traversed_path

In [9]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
goal_flag, path, inital_state_list = local_beam_search(start_list, goal, k, transition_matrix, 1000)
if goal_flag:
    print('cost from start states: ', inital_state_list, ' to goal state ', goal,
          ' is: ', path[make_key(goal)][1])
    traversed_path = get_traversed_path(path, goal)

    print(traversed_path)
else:
    print('goal not found')
    raise AssertionError


inital states randomised with given k= 3  :  [[0, 2], [1, 0], [0, 0]]
goal_state:  [9, 6]
cost from start states:  [[0, 2], [1, 0], [0, 0]]  to goal state  [9, 6]  is:  17
[[1, 0], [2, 1], [3, 2], [4, 2], [5, 2], [6, 2], [7, 2], [8, 2], [9, 3], [8, 4], [7, 4], [6, 4], [5, 5], [6, 6], [6, 7], [7, 8], [8, 7], [9, 6]]


### 5.	Print the algorithm spacetime complexity

In [10]:
#Code Block : Print the Time & Space complexity of algorithm 1
print('Time complexity of local beam search: O(transition_matrix_edge^2 * (k * log k)) ')
print('Space complexity of local beam search: O(transition_matrix_edge^2 + k) ')

Time complexity of local beam search: O(transition_matrix_edge^2 * (k * log k)) 
Space complexity of local beam search: O(transition_matrix_edge^2 + k) 


### 6.	For local search interpret the significance of the hyper parameters if any applicable

k is the hyperparameter, along with seed in this case

1. k = 1 makes it hill climbing and k= inf makes it bfs, in general as k increases we can keep more nodes in memory and find better estimates locally
2. Since we have 5 inital states and k=3, we randomly choose any one out of possible 5C3 combinations. Different set of inital states might give a different optimal route cost and path.